# Lineare Regression

Lineare Regressionsmodelle sind ein guter Ausgangspunkt für Regressionsaufgaben.

Solche Modelle sind beliebt, weil sie sehr schnell angepasst werden können und sehr gut interpretierbar sind. 

Die einfachste Form eines linearen Regressionsmodells ist die Anpassung einer geraden Linie an Daten, wir werden nachher aber auch sehen, wie man kompliziertere Modelle anpassen kann.

Wir beginnen mit den Standard-Importen:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np

plt.rcParams["figure.figsize"] = (10,8)

## Einfache lineare Regression

Eine Gerade ist durch die Gleichung

$y = b + ax$

gegeben, wobei $a$ die Steigung und $b$ als y-Achsenabschnitt bezeichnet wird.

Generieren wir uns ein paar Daten, die um die Gerade $y = -5 + 2x$ gestreut sind:

In [ ]:
m = 100

rng = np.random.default_rng(1)
x = 10 * rng.random(m)
y = 2 * x - 5 + rng.standard_normal(m)
sns.scatterplot(x, y);

Wir können Scikit-Learns `LinearRegression Estimator` verwenden, um die beste Gerade zu ermitteln:

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression(fit_intercept=True)

model.fit(x[:, np.newaxis], y)

xfit = np.linspace(0, 10, 1000)
yfit = model.predict(xfit[:, np.newaxis])

sns.scatterplot(x, y)
sns.lineplot(xfit, yfit);


Die Steigung und der Intercept der Daten sind in den Anpassungsparametern des
Modells enthalten, die in Scikit-Learn immer durch einen abschließenden Unterstrich
gekennzeichnet sind. Hier sind die relevanten Parameter `coef_` und `intercept_`:

In [ ]:
print("Modell: y = {} + {} * x".format(model.intercept_, model.coef_[0]))

Wir sehen, dass die Ergebnisse sehr nahe an den Inputs liegen.

Der LinearRegression-Schätzer ist jedoch noch viel leistungsfähiger.
Er kann neben einfachen Geradengleichungen auch mehrdimensionale lineare Modelle
der Form

$y=a_0+a_1x_1+a_2x_2+\dots$

mit höher-dimensionalen x ermitteln.

Geometrisch entspricht dies der Anpassung einer Ebene an Punkte in drei Dimensionen
oder der Anpassung einer Hyperebene an Punkte in höheren Dimensionen.

Die mehrdimensionale Natur solcher Regressionen macht es schwieriger, sie zu
visualisieren, aber wir können eine dieser Anpassungen in Aktion sehen, indem wir
einige Beispieldaten unter Verwendung des Matrixmultiplikationsoperators von NumPy
erstellen:

In [ ]:
rng = np.random.default_rng(1)
X = rng.random((4, 3))
y = 0.5 + np.dot(X, [1.5, -2., 1.])

model.fit(X, y)
print(model.intercept_)
print(model.coef_)

Hier werden die $y$-Daten aus 4 zufälligen 3-dimensionalen $x$-Werten konstruiert, und die lineare
Regression gewinnt die Koeffizienten zurück, die zur Konstruktion der Daten verwendet wurden.

Auf diese Weise können wir den einzelnen LinearRegression-Schätzer verwenden, um
Linien, Ebenen oder Hyperebenen an unsere Daten anzupassen. Es scheint immer noch,
dass dieser Ansatz auf streng lineare Beziehungen zwischen Variablen beschränkt wäre,
aber es stellt sich heraus, dass wir auch dies lockern können.

## Basisfunktion Regression

Ein Trick, mit dem Sie die lineare Regression an nicht-lineare Beziehungen zwischen
Variablen anpassen können, besteht darin, die Daten entsprechend den Basisfunktionen
zu transformieren.

Die Idee besteht darin, unser mehrdimensionales lineares Modell zu nehmen

$y=a_0+a_1x_1+a_2x_2+a_3x_3+\dots$

und die $x_1, x_2, x_3,$ usw. aus unserer jetzt wieder ein-dimensionalen Eingabe $x$ aufzubauen.
Dazu lassen wir

$x_n = f_n(x)$

sein, wobei $f_n()$ eine Funktion ist, die unsere Daten transformiert.

Wenn zum Beispiel $f_n(x) = x^n$ ist, wird unser Modell zu einer Polynom-Regression:

$y=a_0+a_1x+a_2x^2+a_3x^3+\dots$

Beachten Sie, dass es sich nach wie vor um ein lineares Modell handelt - die
Linearität bezieht sich auf die Tatsache, dass die Koeffizienten $a_n$ sich niemals
miteinander multiplizieren oder dividieren. Was wir effektiv getan haben, ist,
unsere eindimensionalen $x$-Werte in eine höhere Dimension zu projizieren, so
dass eine lineare Anpassung kompliziertere Beziehungen zwischen $x$ und $y$
einpassen kann.

## Polynomische Basisfunktionen

Diese Polynomprojektion ist so nützlich, dass sie mit Hilfe des
`PolynomialFeatures-Transformators` in Scikit-Learn eingebaut ist:

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

x = np.array([2, 3, 4])
poly = PolynomialFeatures(3, include_bias=False)
poly.fit_transform(x[:, None])

Wir sehen hier, dass der Transformator unser eindimensionales Array in ein
dreidimensionales Array umgewandelt hat, indem er den Exponenten jedes Wertes
genommen hat.

Diese neue, höherdimensionale Datendarstellung kann dann in eine lineare Regression
gesteckt werden. Der sauberste Weg, dies zu erreichen, ist die Verwendung einer sogenannten `Pipeline`.

Erstellen wir auf diese Weise ein Polynom-Modell 7. Grades:

In [ ]:
from sklearn.pipeline import make_pipeline
poly_model = make_pipeline(PolynomialFeatures(7), LinearRegression())

Mit dieser Transformation können wir das lineare Modell verwenden, um viel
kompliziertere Beziehungen zwischen $x$ und $y$ anzupassen. Hier ist zum Beispiel
eine verrauschte Sinuswelle:

In [ ]:
rng = np.random.default_rng(1)
x = 10 * rng.random(50)
y = np.sin(x) + 0.1 * rng.standard_normal(50)

poly_model.fit(x[:, np.newaxis], y)
yfit = poly_model.predict(xfit[:, np.newaxis])
plt.subplots(figsize=(9,6))
plt.scatter(x, y)
plt.plot(xfit, yfit);

## Gaußsche Basisfunktionen

Natürlich sind auch andere Basisfunktionen möglich. Ein nützliches Muster ist zum
Beispiel die Anpassung eines Modells, das nicht eine Summe von Polynom-Basen,
sondern eine Summe von Gaußschen Basen ist.
Eine Gaußsche Basis sieht folgendermaßen aus:

In [ ]:
from scipy.stats import norm

x_all = np.arange(-10, 10, 0.001) # entire range of x, both in and out of spec
# mean = 0, stddev = 1, since Z-transform was calculated
y2 = norm.pdf(x_all,0,1)

fig, ax = plt.subplots(figsize=(9,6))
plt.style.use('fivethirtyeight')
ax.plot(x_all,y2)
ax.set_xlim([-4,4])
ax.set_yticklabels([])
ax.set_title('Normal Gaussian Curve')

plt.show()

 Diese Gaußschen Basisfunktionen sind nicht in Scikit-Learn eingebaut, aber wir
 können einen benutzerdefinierten Transformator schreiben, der sie erzeugt.

 Dies ist in der folgenden Abbildung veranschaulicht.

 (Scikit-Learn-Transformatoren sind als Python-Klassen implementiert; der Source-Code
 ist eine gute Quelle, um zu sehen, wie sie erzeugt werden können)

In [ ]:
import Gauss

gauss_model = make_pipeline(Gauss.GaussianFeatures(20),
                            LinearRegression())
gauss_model.fit(x[:, np.newaxis], y)
yfit = gauss_model.predict(xfit[:, np.newaxis])

plt.scatter(x, y)
plt.plot(xfit, yfit)
plt.xlim(0, 10);

Wir stellen dieses Beispiel hier nur an, um deutlich zu machen, dass polynome
Basisfunktionen nichts Magisches an sich haben: Wenn Sie eine Art Intuition in den
Generierungsprozess Ihrer Daten haben, die Sie glauben lässt, dass die eine oder
andere Basis angemessen sein könnte, können Sie sie ebenfalls verwenden.
